In [1]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join
from cobra.io import write_sbml_model

In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#skip
relative_directory = os.getcwd()
print(relative_directory)

data_dir = "/Users/lizrad/Documents/Vibrio_folder"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("GCF")))
model_orig=cobra.io.read_legacy_sbml(join(data_dir, "GCF_001456255.1_rast_metabolic_model.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
model_orig

In [3]:

data_dir = "/Users/lizrad/Dev/iVnat"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("iVnat")))
model=cobra.io.read_sbml_model(join(data_dir, "iVnat.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
model




files found: 
iVnat.xml, iVnat_mapped.xml


Name,GCF_001456255_1_rast_metabolic_model
Memory address,0x01142acd30
Number of metabolites,1842
Number of reactions,1906
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol_0, Extracellular_0"


### Reading the reac_prop

In [ ]:
reac_prop = read_csv("/Users/lizrad/Dev/iVnat/reac_prop.tsv" , sep='\t', skiprows=365)
reac_prop_reordered = reac_prop.set_index("#MNX_ID")
reac_prop_dict = reac_prop_reordered.to_dict('index')

### Reading the reac_xref

In [4]:
reac_xref = read_csv("/Users/lizrad/Dev/iVnat/reac_xref.tsv" , sep='\t')
reac_xref

,XREF,XREF_ID,MNX_ID
0,BIOMASS_EXT,MNXR94667,NaN
1,seed,rxn12985,MNXR94667
2,metanetx,MNXR94667,MNXR94667
3,metanetx,MNXR01,MNXR01
4,bigg,10FTHF5GLUtl,MNXR94668
5,metanetx,MNXR94668,MNXR94668
6,bigg,10FTHF5GLUtm,MNXR94668
7,bigg,10FTHF6GLUtl,MNXR94669
8,metanetx,MNXR94669,MNXR94669
9,bigg,10FTHF6GLUtm,MNXR94669


### Number of seed reactions in the model

In [5]:
print ('This is the total amount of reactions in the model: %i' % (len(model.reactions)))

print ('We focus on mapping those %i reactions in the model that have an SEED ID' % (len([met for met in model.reactions if met.id.startswith('rxn') ])))

for met in model.reactions:
    if not met.id.startswith('rxn'):
        print(met.id)

This is the total amount of reactions in the model: 1906
We focus on mapping those 1776 reactions in the model that have an SEED ID
bio1
EX_cpd00067_e0
EX_cpd00182_e0
EX_cpd00106_e0
EX_cpd00075_e0
EX_cpd00064_e0
EX_cpd08636_c0
EX_cpd00036_e0
EX_cpd11576_e0
EX_cpd04097_e0
EX_cpd11585_e0
EX_cpd00412_e0
EX_cpd11582_e0
EX_cpd00012_e0
EX_cpd11586_e0
EX_cpd01030_e0
EX_cpd00129_e0
EX_cpd00971_e0
EX_cpd01012_e0
EX_cpd00635_e0
EX_cpd00246_e0
EX_cpd00276_e0
EX_cpd00082_e0
EX_cpd00001_e0
EX_cpd00023_e0
EX_cpd00367_e0
EX_cpd00058_e0
EX_cpd00080_e0
EX_cpd00107_e0
EX_cpd00184_e0
EX_cpd00314_e0
EX_cpd01017_e0
EX_cpd00797_e0
EX_cpd00013_e0
EX_cpd00051_e0
EX_cpd10515_e0
EX_cpd00308_e0
EX_cpd00179_e0
EX_cpd00100_e0
EX_cpd11591_e0
EX_cpd00009_e0
EX_cpd00060_e0
EX_cpd00438_e0
EX_cpd00034_e0
EX_cpd00249_e0
EX_cpd00214_e0
EX_cpd00119_e0
EX_cpd11593_e0
EX_cpd00654_e0
EX_cpd00076_e0
EX_cpd00322_e0
EX_cpd01171_e0
EX_cpd00210_e0
EX_cpd15302_c0
EX_cpd15605_e0
EX_cpd00264_e0
EX_cpd00054_e0
EX_cpd00644_e0
EX_cpd00

In [6]:
seed_reactions = [rxn for rxn in model.reactions if rxn.id.startswith('rxn')]

print (len(seed_reactions))
print (len(model.reactions))

1776
1906


In [ ]:
#skip
rxn_mnx2bigg = []
rxn_nobiggid = []

for rxn in [rxn for rxn in cameo_model.reactions if rxn.id.startswith('rxn')]:
    if 'BIGG' in rxn.notes:
        reaction_id = rxn.annotation['BIGG'][0].split(', ')
        tag_split = rxn.id.split('_')
        rxn.id = str(reaction_id[0]) +'_'+ tag_split[1]
        rxn_mnx2bigg.append(reaction_id)
        cameo_model.repair()
    else:
        rxn_nobiggid.append(rxn)
        
print (len(rxn_mnx2bigg))
print (len(rxn_nobiggid))

### Formatting and creating SEED MNX dictionary

In [7]:

df_1 = reac_xref.loc[:,'XREF':'MNX_ID']
groups = df_1.groupby(['MNX_ID','XREF'])
df_1 = groups.apply(lambda x:list(x['XREF_ID']))
df_1 =df_1.unstack('XREF') 
reac_xref_MNX_1_dict = df_1.to_dict('index')
reac_xref_MNX_1_dict

{'MNXR01': {'metanetx': ['MNXR01'],
  'bigg': nan,
  'deprecated': nan,
  'kegg': nan,
  'seed': nan,
  'metacyc': nan,
  'rhea': nan,
  'sabiork': nan,
  'reactome': nan},
 'MNXR100000': {'metanetx': ['MNXR100000'],
  'bigg': ['GALNACT5g'],
  'deprecated': ['MNXR55538'],
  'kegg': nan,
  'seed': nan,
  'metacyc': nan,
  'rhea': nan,
  'sabiork': nan,
  'reactome': nan},
 'MNXR100001': {'metanetx': ['MNXR100001'],
  'bigg': ['GALNTg', 'NAGA2ly'],
  'deprecated': ['MNXR55539', 'MNXR69102', 'MNXR82961'],
  'kegg': nan,
  'seed': nan,
  'metacyc': nan,
  'rhea': nan,
  'sabiork': nan,
  'reactome': nan},
 'MNXR100002': {'metanetx': ['MNXR100002'],
  'bigg': ['GALO'],
  'deprecated': ['MNXR65868', 'MNXR35564'],
  'kegg': ['R01098'],
  'seed': ['rxn00814'],
  'metacyc': nan,
  'rhea': nan,
  'sabiork': nan,
  'reactome': nan},
 'MNXR100003': {'metanetx': ['MNXR100003'],
  'bigg': ['GALOR'],
  'deprecated': ['MNXR31513', 'MNXR2525'],
  'kegg': ['R01095'],
  'seed': ['rxn11552'],
  'metacyc':

In [8]:

reac_property_MNX = read_csv("/Users/lizrad/Documents/Vibrio_folder/reac_xref2.tsv" , sep='\t')
reac_property_MNX.set_index('MNX_ID',inplace = True)
df_3 = reac_property_MNX.fillna('MISSING')  #Fill NA/NaN values using the specified method
df_3
reac_property_MNX_dict = df_3.to_dict('index')
reac_property_MNX_dict

FileNotFoundError: File b'/Users/lizrad/Documents/Vibrio_folder/reac_xref2.tsv' does not exist

In [ ]:
#skip
rxn_ID_not_in_MetaNetX_2 = [rxn for rxn in rxn_nobiggid if rxn.id.startswith('MNXR') and ''.join(re.findall('(MNXR\d*)_',rxn.id)) not in reac_xref_MNX_2_dict.keys()]
print (len(rxn_ID_not_in_MetaNetX_2))

In [ ]:
# Get a mapping from KEGG to the new MNXR IDs, for those reactions that cannot be found in the MNX Database,
# but that have a KEGG ID in their annotation. 
#153644- 170637
df_2 = reac_xref.loc[153642:170637,'XREF_ID':'MNX_ID']
df_2 = df_2.set_index(['XREF_ID'])
seed_2_mnx_dict_2 = df_2.to_dict('index')
seed_2_mnx_dict_2

In [ ]:
reac_xref_MNX_seed = read_csv("/Users/lizrad/Documents/Vibrio_folder/reac_xref2.tsv" , sep='\t', low_memory=False)
df_seed = reac_xref_MNX_seed.loc[153642:170635,'XREF':'MNX_ID']
pivoted_seedd = df_seed.pivot_table(index='XREF',columns='XREF_ID',values='MNX_ID',aggfunc = lambda x: x)
#pivoted_seedd=pivoted_seed.drop(['metanetx'])
pivoted_seedd
reac_xref_MNX_seed_dict = pivoted_seedd.to_dict('index')
reac_xref_MNX_seed_dict2 = reac_xref_MNX_seed_dict['seed']
reac_xref_MNX_seed_dict2

In [ ]:
not_in_reac_xref = []
in_reac_xref = []

for met in model.reactions:
    split=met.id.split("_")
    met_clean=split[0]
    print(met_clean)
    if not met_clean == '':
        
        if not met_clean in reac_xref_MNX_seed_dict2.keys():
            not_in_reac_xref.append(met)
        else:
            in_reac_xref.append(met)

print ('%i metabolites with SEED ID cannot be found in the MNX database' % (len(not_in_reac_xref)))


### SEED Ids to reaction annotation

In [ ]:
for reac in model.reactions:
    if reac.id.startswith('rxn'):
        split, compartment=reac.id.split("_")
        reac.annotation['seed.compound']=split
        print(reac.annotation)
        compartment
        

In [ ]:
for reac in model.reactions:
    data=''
    if 'seed.compound' in reac.annotation:
        data=reac.annotation['seed.compound'] 
        
        for xref in reac_xref_MNX_seed_dict2.keys():
            if xref in data:
                reac.annotation['metanetx.reaction']=reac_xref_MNX_seed_dict2[xref]
                print(reac.annotation)

### Check for multiple ids 

In [ ]:
dupl_seed = {}
for reac in model.reactions:
    if "seed.compound" not in reac.annotation.keys():  #skip EX reactions
        #print(reac.id)
        continue
    if reac.annotation["seed.compound"] not in dupl_seed.keys():
        dupl_seed.setdefault(reac.annotation["seed.compound"],[reac.id]) #similar to get(), but will set dict[key]=default if key is not already in dict.


    else:
        dupl_seed[reac.annotation["seed.compound"]].append(reac.id)
        
dupl_seed  


In [ ]:
dupl_seed

In [ ]:
model.reactions.EX_cpd00067_e0.annotation

In [ ]:
dupl_mnxr = {}
for reac in model.reactions:
    if "metanetx.reaction" not in reac.annotation.keys():
        #print(reac.id)
        continue
        
    if reac.annotation["metanetx.reaction"] not in dupl_mnxr.keys():
            dupl_mnxr.setdefault(reac.annotation["metanetx.reaction"],[reac.id])
    else:
            dupl_mnxr[reac.annotation["metanetx.reaction"]].append(reac.id)
            
           

In [9]:
for ls in dupl_mnxr.values():
    if len(ls) > 1:
        print(ls)

NameError: name 'dupl_mnxr' is not defined

In [10]:
for ls in dupl_mnxr.values():
    if len(ls) > 1:
        print(ls)
        x=[]
        y=[]
        for members in ls: 
            main_id, comp =members.split("_")
            x.append(main_id)
            y.append(comp)
        if not x[0]==x[1]:
            print(x,y)
            for pair in zip(x,y):
                print(len(model.reactions.get_by_id(pair[0]+"_"+pair[1]).reaction))

NameError: name 'dupl_mnxr' is not defined

In [11]:
model.reactions.rxn03023_c0.reactants
model.reactions.rxn03932_c0.reactants

model.reactions.rxn03023_c0.annotation

model.reactions.rxn03932_c0.reaction


[<Metabolite atp_c0 at 0x115b70198>, <Metabolite cpd02446_c0 at 0x115c717b8>]

[<Metabolite atp_c0 at 0x115b70198>, <Metabolite d5kg_c0 at 0x115d1b320>]

{}

'atp_c0 + d5kg_c0 <=> MNXM91569_c0 + adp_c0 + h_c0'

In [12]:
for reac in model.reactions:
    
    
    if "metanetx.reaction" not in reac.annotation.keys():
        #print(reac.id)
        continue
    if reac.id.startswith('rxn'):    
        if reac.id in ["rxn03023_c0"]:
            pass
        else:
            split, compartment=reac.id.split("_")
            reac.id
            reac.id=reac.annotation['metanetx.reaction']+ '_' + compartment
            model.repair()
            print(reac.id)
    

In [ ]:
#chem_xref['#XREF'] = chem_xref['#XREF'].str.replace(':','\t')

#^[0-9A-Za-z]*:[0-9A-Za-z]*
#    (^[0-9A-Za-z]*):([0-9A-Za-z]*)
#        $1 \t $2
        



In [13]:
write_sbml_model(model, "/Users/lizrad/Dev/iVnat/iVnat.xml")